In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import joblib

kaggle_input="./kaggle/input"
# kaggle_working="./kaggle/working"
kaggle_temp="./kaggle/temp1"

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

In [2]:
f_train=pd.read_csv(f"{kaggle_temp}/f_train.csv")
# f_test=pd.read_csv(f"{kaggle_working}/f_test.csv")


In [3]:
t_sample=len(f_train)
f_train=f_train[:t_sample]
f_train

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,...,RP_19.92savg55,RP_19.92savg56,RP_19.92savg57,RP_19.92savg58,RP_19.92savg59
0,1.0,0.0,0.0,0.0,0.0,...,0.132000,0.166000,0.128000,0.136000,0.212000
1,1.0,0.0,0.0,0.0,0.0,...,0.132000,0.166000,0.128000,0.136000,0.212000
2,1.0,0.0,0.0,0.0,0.0,...,0.132000,0.166000,0.128000,0.136000,0.212000
3,1.0,0.0,0.0,0.0,0.0,...,0.132000,0.166000,0.128000,0.136000,0.212000
4,1.0,0.0,0.0,0.0,0.0,...,0.132000,0.166000,0.128000,0.136000,0.212000
...,...,...,...,...,...,...,...,...,...,...,...
67995,1.0,0.0,0.0,0.0,0.0,...,3.352000,2.532000,1.470000,2.022000,1.590000
67996,0.0,0.0,0.0,0.0,0.0,...,0.034000,0.218000,1.174000,2.354000,0.034000
67997,0.0,0.0,0.0,0.0,1.0,...,2072.209961,2065.580005,2200.430029,2051.482007,2411.711987
67998,0.0,0.0,0.0,0.0,1.0,...,2072.209961,2065.580005,2200.430029,2051.482007,2411.711987


In [4]:
f_train.fillna(method='bfill',inplace=True)
f_train.isna().sum()


seizure_vote      0
lpd_vote          0
gpd_vote          0
lrda_vote         0
grda_vote         0
                 ..
RP_19.92savg55    0
RP_19.92savg56    0
RP_19.92savg57    0
RP_19.92savg58    0
RP_19.92savg59    0
Length: 4806, dtype: int64

In [5]:
import time

f_train
vote_cols=['seizure_vote','lpd_vote','gpd_vote','lrda_vote','grda_vote','other_vote']
train_cols=[col for col in f_train.columns if col not in vote_cols]
vote_cols=vote_cols[:]
models=[RandomForestRegressor(n_estimators=3000, random_state=3,n_jobs=-1, min_samples_split=2) for i in range(len(vote_cols))]
for i in range(len(vote_cols)):
    start_time = time.time()
    
    print(f'training {vote_cols[i]} model')
    models[i].fit(f_train[train_cols],f_train[vote_cols[i]])
    joblib.dump(models[i],f'{kaggle_temp}/models/{vote_cols[i]}.joblib')
    print(f'trained and saved at {kaggle_temp}/models/{vote_cols[i]}.joblib')

    print("--- %s seconds --- for %s model" % (time.time() - start_time,vote_cols[i]))



training gpd_vote model
trained and saved at ./kaggle/temp1/models/gpd_vote.joblib
--- 1543.0944006443024 seconds --- for gpd_vote model
training lrda_vote model
trained and saved at ./kaggle/temp1/models/lrda_vote.joblib
--- 1338.3651518821716 seconds --- for lrda_vote model
training grda_vote model


In [ ]:
p_votes=pd.DataFrame(columns=vote_cols)
for i in range(len(models)):
    p_votes[vote_cols[i]]=models[i].predict(f_train[train_cols])

p_votes

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
0,1.0,0.0,0.000000,0.0,0.000000,0.00000
1,1.0,0.0,0.000000,0.0,0.000000,0.00000
2,1.0,0.0,0.000000,0.0,0.000000,0.00000
3,1.0,0.0,0.000000,0.0,0.000000,0.00000
4,1.0,0.0,0.000000,0.0,0.000000,0.00000
...,...,...,...,...,...,...
11495,0.0,0.0,0.041975,0.0,0.041975,0.91605
11496,0.0,0.0,0.041975,0.0,0.041975,0.91605
11497,0.0,0.0,0.041975,0.0,0.041975,0.91605
11498,0.0,0.0,0.041975,0.0,0.041975,0.91605


In [ ]:
# generalizing probs
for i in range(100):
   sum_votes=sum(p_votes.loc[i,vote_cols])
   p_votes.loc[i,vote_cols]=p_votes.loc[i,vote_cols].apply(lambda x: (x/sum_votes) )
p_votes.loc[30:31,:]

,seizure_vote,lpd_vote,gpd_vote,lrda_vote,grda_vote,other_vote
30,0.716792,0.0,0.283208,0.0,0.0,0.0
31,0.716792,0.0,0.283208,0.0,0.0,0.0
